In [1]:
import org.apache.spark.sql.{SQLContext, SaveMode}
import org.apache.spark.sql.types.{StructType, StructField, StringType, DoubleType};
import com.databricks.spark.xml._

val sqlContext = new SQLContext(sc)
import sqlContext.implicits._
sqlContext.sql("set spark.sql.caseSensitive=true") 
spark.sparkContext.uiWebUrl

Waiting for a Spark session to start...

sqlContext = org.apache.spark.sql.SQLContext@47c0373b


Some(http://iuni2:4040)

In [2]:
//val sqlContext = new SQLContext(sc)
val df = sqlContext.read
  .format("com.databricks.spark.xml")
  .option("rowTag", "REC")
  //.schema(customSchema)
  .load("/dataRCSC/*.xml")

df.show(2)
df.printSchema()

Name: org.apache.spark.SparkException
Message: Job 0 cancelled as part of cancellation of all jobs
StackTrace:   at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1890)
  at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1825)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$doCancelAllJobs$1.apply$mcVI$sp(DAGScheduler.scala:833)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$doCancelAllJobs$1.apply(DAGScheduler.scala:833)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$doCancelAllJobs$1.apply(DAGScheduler.scala:833)
  at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
  at org.apache.spark.scheduler.DAGScheduler.doCancelAllJobs(DAGScheduler.scala:833)
  at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:2083)
  at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2060)
 

In [3]:
val WoS2 = df.select("UID","dynamic_data.cluster_related.identifiers.identifier","static_data.summary.doctypes.doctype",
                      "static_data.summary.names.name","static_data.summary.titles","static_data.summary.conferences.conference",
                      "static_data.fullrecord_metadata.addresses.address_name", "static_data.fullrecord_metadata.reprint_addresses",
                      "static_data.fullrecord_metadata.abstracts.abstract.abstract_text",
                      "static_data.fullrecord_metadata.keywords.keyword","static_data.fullrecord_metadata.references.reference",
                      "static_data.contributors.contributor","static_data.summary.EWUID",
                      "static_data.summary.pub_info","static_data.summary.publishers","static_data.fullrecord_metadata.fund_ack")
WoS2.printSchema

Name: Unknown Error
Message: lastException: Throwable = null
<console>:35: error: not found: value df
       val WoS2 = df.select("UID","dynamic_data.cluster_related.identifiers.identifier","static_data.summary.doctypes.doctype",
                  ^

StackTrace: 

In [7]:
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Row, SQLContext}
import org.apache.spark.sql.functions.countDistinct
import org.apache.spark.sql.SaveMode

//WoS2.write.mode(SaveMode.Overwrite).json("/dataRCSC/WoS_json")
val WoSref = WoS2.select("UID","reference.uid").toDF("citing","refList").withColumn("cited", explode(col("refList")))
WoSref.select("citing","cited").orderBy($"cited".desc).coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true").option("compression", "gzip")
    .save("/dataRCSC/WoS_json/citeEdges.csv/")

WoSref = [citing: string, refList: array<string> ... 1 more field]


[citing: string, refList: array<string> ... 1 more field]

In [6]:
val WoSref = WoS2.select("UID","reference.uid").toDF("citing","refList").withColumn("cited", explode(col("refList")))
WoSref.select("citing","cited").coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true").option("compression", "gzip")
    .save("/dataRCSC/WoS_json/refEdges.csv")

WoSref = [citing: string, refList: array<string> ... 1 more field]


[citing: string, refList: array<string> ... 1 more field]

In [42]:
val WoS = spark.read.format("parquet").load("/WoSraw/parquet/*.parquet")

val WoS2 = WoS.select("UID","static_data.summary.titles","static_data.fullrecord_metadata.category_info","static_data.summary.names.name")
WoS2.printSchema

root
 |-- UID: string (nullable = true)
 |-- titles: struct (nullable = true)
 |    |-- _count: long (nullable = true)
 |    |-- title: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |-- _translated: string (nullable = true)
 |    |    |    |-- _type: string (nullable = true)
 |-- category_info: struct (nullable = true)
 |    |-- headings: struct (nullable = true)
 |    |    |-- _count: long (nullable = true)
 |    |    |-- heading: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |-- subheadings: struct (nullable = true)
 |    |    |-- _count: long (nullable = true)
 |    |    |-- subheading: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |-- subjects: struct (nullable = true)
 |    |    |-- _count: long (nullable = true)
 |    |    |-- subject: array (nullable = true)
 |    |    |    |-- element: struct (con

WoS = [UID: string, _r_id_disclaimer: string ... 13 more fields]
WoS2 = [UID: string, titles: struct<_count: bigint, title: array<struct<_VALUE:string,_translated:string,_type:string>>> ... 2 more fields]


[UID: string, titles: struct<_count: bigint, title: array<struct<_VALUE:string,_translated:string,_type:string>>> ... 2 more fields]

In [51]:
val WoS3 = WoS2.select("UID","name.first_name").withColumn("firstCombo", explode(col("first_name")))
val firstCount = WoS3.groupBy("firstCombo").agg(count("*") as "comboCount", countDistinct("UID") as "paperCount")
firstCount.show

+------------+----------+----------+
|  firstCombo|comboCount|paperCount|
+------------+----------+----------+
|           K|   3073292|   2611856|
|          MM|    240812|    238666|
|          LT|     45142|     45037|
|          DZ|     13835|     13791|
|  Michael C.|     18552|     18495|
|    Monalisa|       578|       578|
|        Soni|       281|       281|
|Douglas Cole|         8|         8|
|       E. S.|     21086|     20940|
|       D. E.|     20047|     19704|
|    W. Allen|       183|       183|
|      Helmut|     19644|     19491|
|    Francois|     56233|     54893|
|        Ritu|      4079|      4068|
|    Yasunari|      2169|      2168|
|    Noriyuki|     10453|     10415|
|    Julianne|      1783|      1783|
|      Hideki|     30011|     29517|
|    Byung J.|        42|        42|
| Jules P. P.|        74|        74|
+------------+----------+----------+
only showing top 20 rows



WoS3 = [UID: string, first_name: array<string> ... 1 more field]
firstCount = [firstCombo: string, comboCount: bigint ... 1 more field]


[firstCombo: string, comboCount: bigint ... 1 more field]

In [52]:
firstCount.coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true")
    .option("quote", "\"")
    .save("wasbs://mag-2020-09-01@iunimag.blob.core.windows.net/2020-08-03/nameCount.csv")

In [7]:
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Row, SQLContext}
import org.apache.spark.sql.functions.countDistinct
import org.apache.spark.sql.SaveMode

val session = SparkSession.builder().getOrCreate()

val azureFile = sc.textFile("/azureBlob.txt")
session.conf.set(
    "fs.azure.account.key.iunimag.blob.core.windows.net",
     azureFile.collect().mkString
)
val sdf = spark.read.format("csv")
    .option("header", "true")
    .option("quote", "\"")
    .option("escape", "\"")
    //.option("inferSchema","true")
    //.load("/dataRCSC/metaLitCovid-09-13.csv")
    .load("wasbs://mag-2020-09-01@iunimag.blob.core.windows.net/2020-08-03/MAGInstTable.csv")
sdf.describe().filter($"summary" === "count").show

+-------+-------------+-----------+--------------+----+----------+-------------+--------+---------+------+
|summary|affiliationID|displayName|normalizedName|wiki|paperTotal|citationTotal|Latitude|Longitude|GRIDid|
+-------+-------------+-----------+--------------+----+----------+-------------+--------+---------+------+
|  count|         9210|       9210|          9210|9119|      9210|         9210|    8984|     9030|  7856|
+-------+-------------+-----------+--------------+----+----------+-------------+--------+---------+------+



session = org.apache.spark.sql.SparkSession@255044a5
azureFile = /azureBlob.txt MapPartitionsRDD[59] at textFile at <console>:58
sdf = [affiliationID: string, displayName: string ... 7 more fields]


[affiliationID: string, displayName: string ... 7 more fields]

In [10]:
//sdf.printSchema
//val magAffiliation2 = magAffiliation.select("_c0","_c2","_c3","_c4","_c6","_c7","_c8","_c9","_c10","_c11","_c12")
//.toDF("AffiliationId2","normalizedName","displayName","GRIDid","wiki","paperTotal","paperFamilyTotal","citationTotal", "Iso3166Code", "Latitude", "Longitude")
//val sdf2 = sdf.drop("displayName","normalizedName","wiki","paperTotal","citationTotal","Latitude","Longitude","GRIDid").join(magAffiliation2, sdf("affiliationID")===magAffiliation2("AffiliationId2"), "left").cache()
sdf2.drop("AffiliationId2").coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true")
    .option("quote", "\"")
    .save("wasbs://mag-2020-09-01@iunimag.blob.core.windows.net/2020-08-03/MAGInstTableUpdate.csv")

In [3]:
val magPapers = spark.read.format("csv")
      //.option("header", "true")
      .option("delimiter","\t")
      .option("inferSchema","true")
      .load("wasbs://mag-2020-09-01@iunimag.blob.core.windows.net/mag/Papers.txt")
magPapers.show

Name: org.apache.spark.SparkException
Message: Job 4 cancelled as part of cancellation of all jobs
StackTrace:   at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1890)
  at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1825)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$doCancelAllJobs$1.apply$mcVI$sp(DAGScheduler.scala:833)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$doCancelAllJobs$1.apply(DAGScheduler.scala:833)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$doCancelAllJobs$1.apply(DAGScheduler.scala:833)
  at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
  at org.apache.spark.scheduler.DAGScheduler.doCancelAllJobs(DAGScheduler.scala:833)
  at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:2083)
  at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2060)
 

In [41]:
val doiMatch = sdf.join(magPapers, sdf("doi")===magPapers("_c2"), "left").cache()
doiMatch.describe().filter($"summary" === "count").show

+-------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+-----+------+-----+-----+-----+-----+-----+-----+---+-----+-----+-----+-----+----+----+----+----+-----+-----+-----+-----+-----+-----+----+
|summary|cord_uid|  sha|source_x| title|   doi|pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|  url| s2_id|  _c0|  _c1|  _c2|  _c3|  _c4|  _c5|_c6|  _c7|  _c8|  _c9| _c10|_c11|_c12|_c13|_c14| _c15| _c16| _c17| _c18| _c19| _c20|_c21|
+-------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+-----+------+-----+-----+-----+-----+-----+-----+---+-----+-----+-----+-----+----+----+----+----+-----+-----+-----+-----+-----+-----+----+
|  count|  128648|55847|  128648|128620|100773|60927|    99169| 128

doiMatch = [cord_uid: string, sha: string ... 40 more fields]


[cord_uid: string, sha: string ... 40 more fields]

In [4]:
val idMap = spark.read.format("csv").option("header", false)
            .load("wasbs://mag-2020-09-01@iunimag.blob.core.windows.net/2020-09-01-CORD-UID-MappedTo-2020-08-24-MAG-ID.csv")
val idMatch = sdf.join(idMap, sdf("cord_uid")===idMap("_c0"), "left").withColumnRenamed("_c1","MAGid").drop("_c0")
idMatch.describe().filter($"summary" === "count").show

+-------+----------+--------+-----+--------+------+------+------+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+--------------+----------+--------------+-----+-----+-----------+------+
|summary|Unnamed: 0|cord_uid|  sha|source_x| title|   doi| pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|   url| s2_id|pmcAuthors|pmcAffiliation|pdfAuthors|pdfAffiliation| pmid| year|annotations| MAGid|
+-------+----------+--------+-----+--------+------+------+------+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+--------------+----------+--------------+-----+-----+-----------+------+
|  count|    244035|  244035|95650|  244035|243977|160186|102577|   140745| 244035|  172927|      243978| 236700| 229504|     0|           69316|    3202|

idMap = [_c0: string, _c1: string]
idMatch = [Unnamed: 0: string, cord_uid: string ... 26 more fields]


lastException: Throwable = null


[Unnamed: 0: string, cord_uid: string ... 26 more fields]

In [5]:
val idMatch2 = idMatch.groupBy("cord_uid").agg(concat_ws(";", collect_list("MAGid")) as "MAGids")
idMatch2.describe().filter($"summary" === "count").show

+-------+--------+------+
|summary|cord_uid|MAGids|
+-------+--------+------+
|  count|  230834|230834|
+-------+--------+------+



idMatch2 = [cord_uid: string, MAGids: string]


[cord_uid: string, MAGids: string]

In [5]:
val magMatch = idMatch.join(magPapers, idMatch("MAGid")===magPapers("_c0"), "left").cache()
magMatch.describe().filter($"summary" === "count").show

Name: Compile Error
Message: <console>:44: error: not found: value magPapers
       val magMatch = idMatch.join(magPapers, idMatch("MAGid")===magPapers("_c0"), "left").cache()
                                   ^
<console>:44: error: not found: value magPapers
       val magMatch = idMatch.join(magPapers, idMatch("MAGid")===magPapers("_c0"), "left").cache()
                                                                 ^

StackTrace: 

In [4]:
val magPPA = spark.read.format("csv")
      //.option("header", "true")
      .option("delimiter","\t")
      .option("inferSchema","true")
      .load("wasbs://mag-2020-09-01@iunimag.blob.core.windows.net/mag/PaperAuthorAffiliations.txt")

magPPA.show

+---+----------+----------+---+--------------------+--------------------+
|_c0|       _c1|       _c2|_c3|                 _c4|                 _c5|
+---+----------+----------+---+--------------------+--------------------+
|  9|2632942543|      null|  1|Victoriano Perruc...|                null|
| 15| 199142497|      null|  1|     Robert Münscher|Heidelberg, Deuts...|
| 15| 680395887|      null|  2|       Julia Hormuth|Reutlingen, Deuts...|
| 23|1243978490|  79576946|  1|     Eric T Stoopler|Department of Ora...|
| 23|2582258949|      null|  3| Arthur S Kuperstein|                null|
| 23|2582736345|      null|  2|        Ying Wai Sia|                null|
| 58|2641488431|      null|  1|             I. Uray|                null|
| 58|2693978806|      null|  2|           P. Kovacs|                null|
| 79|2662843304|      null|  1|           K. Suresh|                null|
| 79|2683440697|      null|  2|     T. S. J. Norman|                null|
| 87|2676159639|      null|  1| Gabrie

magPPA = [_c0: bigint, _c1: bigint ... 4 more fields]


[_c0: bigint, _c1: bigint ... 4 more fields]

In [7]:
val magPPA2 = magPPA.toDF("MAGid2","authorID","affiliationID","authorOrder","author","affiliation").filter("affiliationID is not null")
val magMatchPPA = idMatch.join(magPPA2, idMatch("MAGid")===magPPA2("MAGid2"), "left").cache()
magMatchPPA.describe().filter($"summary" === "count").show

+-------+----------+--------+------+--------+------+------+------+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+--------------+----------+--------------+------+------+-----------+------+------+--------+-------------+-----------+------+-----------+
|summary|Unnamed: 0|cord_uid|   sha|source_x| title|   doi| pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|   url| s2_id|pmcAuthors|pmcAffiliation|pdfAuthors|pdfAffiliation|  pmid|  year|annotations| MAGid|MAGid2|authorID|affiliationID|authorOrder|author|affiliation|
+-------+----------+--------+------+--------+------+------+------+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+--------------+----------+--------------+------+------+-----------+------+------+--------+--------

magPPA2 = [MAGid2: bigint, authorID: bigint ... 4 more fields]
magMatchPPA = [Unnamed: 0: string, cord_uid: string ... 32 more fields]


[Unnamed: 0: string, cord_uid: string ... 32 more fields]

In [6]:
val magAffiliation = spark.read.format("csv")
      //.option("header", "true")
      .option("delimiter","\t")
      .option("inferSchema","true")
      .load("wasbs://mag-2020-09-01@iunimag.blob.core.windows.net/mag/Affiliations.txt")

val magAffiliation2 = magAffiliation.select("_c0","_c2","_c3","_c4","_c6","_c7","_c8","_c9","_c10","_c11","_c12").toDF("AffiliationId2","normalizedName","displayName","GRIDid","wiki","paperTotal","paperFamilyTotal","citationTotal", "Iso3166Code", "Latitude", "Longitude")
magAffiliation2.describe().show

+-------+--------------------+--------------------+--------------------+-------------+--------------------+------------------+------------------+-----------------+-----------+------------------+------------------+
|summary|      AffiliationId2|      normalizedName|         displayName|       GRIDid|                wiki|        paperTotal|  paperFamilyTotal|    citationTotal|Iso3166Code|          Latitude|         Longitude|
+-------+--------------------+--------------------+--------------------+-------------+--------------------+------------------+------------------+-----------------+-----------+------------------+------------------+
|  count|               25785|               25785|               25785|        19026|               25496|             25785|             25785|            25785|      24689|             24790|             24790|
|   mean| 1.076605067649137E9|                null|                null|         null|                null|3316.6961023851077|3263.5179367849523

magAffiliation = [_c0: bigint, _c1: int ... 12 more fields]
magAffiliation2 = [AffiliationId2: bigint, normalizedName: string ... 9 more fields]


[AffiliationId2: bigint, normalizedName: string ... 9 more fields]

In [9]:
val magAffiliation2 = magAffiliation.select("_c0","_c2","_c3","_c4","_c6","_c7","_c9","_c10","_c11").toDF("affiliationID2","normalizedName","displayName","GRIDid","wiki","paperTotal", "citationTotal", "Latitude", "Longitude")
val magMatchPPA2 = magMatchPPA.join(magAffiliation2, magMatchPPA("affiliationID")===magAffiliation2("affiliationID2"), "left").cache()
magMatchPPA2.describe().filter($"summary" === "count").show

+-------+----------+--------+------+--------+------+------+------+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+--------------+----------+--------------+------+------+-----------+------+------+--------+-------------+-----------+------+-----------+--------------+--------------+-----------+------+------+----------+-------------+--------+---------+
|summary|Unnamed: 0|cord_uid|   sha|source_x| title|   doi| pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|   url| s2_id|pmcAuthors|pmcAffiliation|pdfAuthors|pdfAffiliation|  pmid|  year|annotations| MAGid|MAGid2|authorID|affiliationID|authorOrder|author|affiliation|affiliationID2|normalizedName|displayName|GRIDid|  wiki|paperTotal|citationTotal|Latitude|Longitude|
+-------+----------+--------+------+--------+------+------+------+---------+-------+--------+------------+--

magAffiliation2 = [affiliationID2: bigint, normalizedName: string ... 7 more fields]
magMatchPPA2 = [Unnamed: 0: string, cord_uid: string ... 41 more fields]


[Unnamed: 0: string, cord_uid: string ... 41 more fields]

In [10]:
val PaperMAG = magMatchPPA2.groupBy("cord_uid").agg(concat_ws(";", collect_list("MAGid2")) as "MAGids",
                                                    concat_ws(";", collect_list("authorID")) as "authorids",
                                                    concat_ws(";", collect_list("authorOrder")) as "authorOrders",
                                                    concat_ws(";", collect_list("affiliationID2")) as "affiliationids",
                                                    concat_ws(";", collect_list("normalizedName")) as "affiliationNormalized",
                                                    concat_ws(";", collect_list("displayName")) as "affiliationNames",
                                                    concat_ws(";", collect_list("GRIDid")) as "GRIDids")
PaperMAG.show(false)

+--------+-------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------+--------------------+-------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------

PaperMAG = [cord_uid: string, MAGids: string ... 6 more fields]


[cord_uid: string, MAGids: string ... 6 more fields]

In [11]:
val PaperMAG2 = sdf.join(PaperMAG, Seq("cord_uid"), "left").drop("_c0").cache()
PaperMAG2.describe().filter($"summary" === "count").show

+-------+--------+----------+-----+--------+------+------+------+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+--------------+----------+--------------+-----+-----+-----------+------+---------+------------+--------------+---------------------+----------------+-------+
|summary|cord_uid|Unnamed: 0|  sha|source_x| title|   doi| pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|   url| s2_id|pmcAuthors|pmcAffiliation|pdfAuthors|pdfAffiliation| pmid| year|annotations|MAGids|authorids|authorOrders|affiliationids|affiliationNormalized|affiliationNames|GRIDids|
+-------+--------+----------+-----+--------+------+------+------+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+--------------+----------+--------------+-----+-----+----

PaperMAG2 = [cord_uid: string, Unnamed: 0: string ... 32 more fields]


[cord_uid: string, Unnamed: 0: string ... 32 more fields]

In [12]:
PaperMAG2.coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true")
    .option("quote", "\"")
    .save("wasbs://mag-2020-09-01@iunimag.blob.core.windows.net/2020-08-03/CORD-19-MAGinstsFull.csv")

In [13]:
val insititutionMAG = magMatchPPA2.groupBy("affiliationID").agg(//concat_ws(";", collect_list("MAGid2")) as "MAGids",
                                                    //concat_ws(";", collect_list("cord_uid")) as "cord_uids",
                                                    first("displayName") as "displayName",
                                                    first("normalizedName") as "normalizedName",
                                                    first("wiki") as "wiki",
                                                    first("paperTotal") as "paperTotal", 
                                                    first("citationTotal") as "citationTotal",
                                                    first("Latitude") as "Latitude",
                                                    first("Longitude") as "Longitude",
                                                    first("GRIDid") as "GRIDid")
insititutionMAG.show(false)

+-------------+---------------------------------------------------+--------------------------------------------------+--------------------------------------------------------------------------------+----------+-------------+--------+-----------+-------------+
|affiliationID|displayName                                        |normalizedName                                    |wiki                                                                            |paperTotal|citationTotal|Latitude|Longitude  |GRIDid       |
+-------------+---------------------------------------------------+--------------------------------------------------+--------------------------------------------------------------------------------+----------+-------------+--------+-----------+-------------+
|746986       |Halmstad University                                |halmstad university                               |http://en.wikipedia.org/wiki/Halmstad_University,_Sweden                        |2548      |39177     

insititutionMAG = [affiliationID: bigint, displayName: string ... 7 more fields]


[affiliationID: bigint, displayName: string ... 7 more fields]

In [14]:
insititutionMAG//.select("wosid","headings","subheadings","WC","SC","disciplines","subdisciplines","fractions")
    .coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true")
    .option("quote", "\"")
    .save("wasbs://mag-2020-09-01@iunimag.blob.core.windows.net/2020-08-03/MAGInstTable.csv")

In [17]:
val meta0b = spark.read.format("csv")
      .option("header", "true")
      //.option("delimiter","\t")
      .option("inferSchema","true")
      .load("/dataRCSC/metaAuthors.csv")

meta0b.describe().filter($"summary" === "count").show

+-------+------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+----------+---------+--------------+--------------+------+
|summary|   _c0|cord_uid|  sha|source_x| title|   doi|pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|   url| s2_id|pmcAuthors|pdfAuthors|SSAuthors|pmcAffiliation|pdfAffiliation|SSRefs|
+-------+------+--------+-----+--------+------+------+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+------+------+----------+----------+---------+--------------+--------------+------+
|  count|137411|  137411|63458|  137411|137370|113076|70527|   100629| 137359|  105932|      137366| 132106| 130231|  2649|           27941|    3719|         64965|         52370|124586|111762|     52315|     61

meta0b = [_c0: int, cord_uid: string ... 24 more fields]


[_c0: int, cord_uid: string ... 24 more fields]

In [23]:
val meta0 = spark.read.format("csv")
    .option("header", "true")
    .option("quote", "\"")
    .option("escape", "\"")
    //.option("inferSchema","true")
    .load("wasbs://mag-2020-09-01@iunimag.blob.core.windows.net/testData.csv")
meta0.describe().filter($"summary" === "count").show

+-------+--------+---+--------+-----+----+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+---+-----+----------+--------------+----------+--------------+----+-----------+------+---------+------------+--------------+---------------------+----------------+-------+--------+------+----+----------+----------+-------------+--------------+-------------+-----------+------------+
|summary|cord_uid|sha|source_x|title| doi|pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|url|s2_id|pmcAuthors|pmcAffiliation|pdfAuthors|pdfAffiliation|pmid|annotations|MAGids|authorids|authorOrders|affiliationids|affiliationNormalized|affiliationNames|GRIDids|xref_doi|art_no| UID|WoSauthors|WoSjournal|score_title_0|score_author_0|ID_researcher|Author_name|author_order|
+-------+--------+---+--------+-----+----+-----+---------+-------+--------+------------+-------+----

meta0 = [cord_uid: string, sha: string ... 40 more fields]


[cord_uid: string, sha: string ... 40 more fields]

In [57]:
meta0.groupBy("cord_uid").count().describe().filter($"summary" === "count").show

+-------+--------+-----+
|summary|cord_uid|count|
+-------+--------+-----+
|  count|    4989| 4989|
+-------+--------+-----+



In [8]:
val SS = spark.read.json("/dataRCSC/SS-2020-05-27/*.gz")
SS.printSchema()

root
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- ids: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- doiUrl: string (nullable = true)
 |-- entities: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- fieldsOfStudy: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- inCitations: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- journalName: string (nullable = true)
 |-- journalPages: string (nullable = true)
 |-- journalVolume: string (nullable = true)
 |-- magId: string (nullable = true)
 |-- outCitations: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- paperAbstract: string (nullable = true)
 |-- pdfUrls: array (nullable = true)
 |    |-- element: string (containsNull = 

SS = [authors: array<struct<ids:array<string>,name:string>>, doi: string ... 19 more fields]


[authors: array<struct<ids:array<string>,name:string>>, doi: string ... 19 more fields]

In [27]:
val SS0 = SS.select("authors.name", "authors.ids", "doi", "id", "inCitations", "magId")
            .toDF("SSauthorNames","SSauthorIDs","SSdoi","SSid","SSrefs","SSmagId").cache()
val meta1 = meta0.join(SS0, meta0("sha")===SS0("SSid"), "left")
meta1.describe().filter($"summary" === "count").show

+-------+--------+---+--------+-----+----+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+---+-----+----------+--------------+----------+--------------+----+-----------+------+---------+------------+--------------+---------------------+----------------+-------+--------+------+----+----------+----------+-------------+--------------+-------------+-----------+------------+-----+----+-------+
|summary|cord_uid|sha|source_x|title| doi|pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|url|s2_id|pmcAuthors|pmcAffiliation|pdfAuthors|pdfAffiliation|pmid|annotations|MAGids|authorids|authorOrders|affiliationids|affiliationNormalized|affiliationNames|GRIDids|xref_doi|art_no| UID|WoSauthors|WoSjournal|score_title_0|score_author_0|ID_researcher|Author_name|author_order|SSdoi|SSid|SSmagId|
+-------+--------+---+--------+-----+----+-----+---------+----

SS0 = [SSauthorNames: array<string>, SSauthorIDs: array<array<string>> ... 4 more fields]
meta1 = [cord_uid: string, sha: string ... 46 more fields]


[cord_uid: string, sha: string ... 46 more fields]

In [51]:
val SSexp = SS0.select(explode($"SSauthorIDs"),size($"SSauthorNames")).withColumn("size", size($"col"))
SSexp.orderBy($"size".desc).show

+-----------+-------------------+----+
|        col|size(SSauthorNames)|size|
+-----------+-------------------+----+
| [51978410]|                  1|   1|
| [71630645]|                  3|   1|
| [52393267]|                  3|   1|
| [72050485]|                  3|   1|
|  [1776025]|                  6|   1|
|  [2836018]|                  6|   1|
| [47422037]|                  6|   1|
|  [1679387]|                  6|   1|
|[145799243]|                  6|   1|
|[145754722]|                  6|   1|
|[114402012]|                  1|   1|
|[136282636]|                  1|   1|
|[146709087]|                  3|   1|
| [51218054]|                  3|   1|
|[144681826]|                  3|   1|
|[120457898]|                  1|   1|
|[152634023]|                  4|   1|
|[153204165]|                  4|   1|
|  [4244638]|                  4|   1|
|  [1796371]|                  4|   1|
+-----------+-------------------+----+
only showing top 20 rows



SSexp = [col: array<string>, size(SSauthorNames): int ... 1 more field]


[col: array<string>, size(SSauthorNames): int ... 1 more field]

In [30]:
val meta2 = meta0.join(SS0, meta0("doi")===SS0("SSdoi"), "left").cache()
meta2.describe().filter($"summary" === "count").show

+-------+--------+---+--------+-----+----+-----+---------+-------+--------+------------+-------+-------+------+----------------+--------+--------------+--------------+---+-----+----------+--------------+----------+--------------+----+-----------+------+---------+------------+--------------+---------------------+----------------+-------+--------+------+----+----------+----------+-------------+--------------+-------------+-----------+------------+-----+----+-------+
|summary|cord_uid|sha|source_x|title| doi|pmcid|pubmed_id|license|abstract|publish_time|authors|journal|mag_id|who_covidence_id|arxiv_id|pdf_json_files|pmc_json_files|url|s2_id|pmcAuthors|pmcAffiliation|pdfAuthors|pdfAffiliation|pmid|annotations|MAGids|authorids|authorOrders|affiliationids|affiliationNormalized|affiliationNames|GRIDids|xref_doi|art_no| UID|WoSauthors|WoSjournal|score_title_0|score_author_0|ID_researcher|Author_name|author_order|SSdoi|SSid|SSmagId|
+-------+--------+---+--------+-----+----+-----+---------+----

meta2 = [cord_uid: string, sha: string ... 46 more fields]


[cord_uid: string, sha: string ... 46 more fields]

In [41]:
SSmatched2.show

+--------+--------------------+--------------------+--------------------+--------------------+
|cord_uid|              SSdois|    SSauthorNameList|      SSauthorIDList|           SSrefList|
+--------+--------------------+--------------------+--------------------+--------------------+
|iqpgmls6|10.1016/j.bbmt.20...|Sakara Hutspardol...|12492852;49771725...|caf95fa590e7cbb34...|
|xe02b8ga|10.1016/j.vaccine...|Markus Czub;Hana ...|4789603;4802727;5...|15b0a02a3a59cdc50...|
|6b35vzd8|10.1016/j.ajic.20...|David M Moore;Bru...|47932570;12068284...|cb8ad6cfaa028e356...|
|n7uzdsx7|10.1016/j.ajic.20...|David M Moore;Bru...|47932570;12068284...|cb8ad6cfaa028e356...|
|xavtxl4m|10.2196/jmir.7.4.e46|Carlos Acosta Riz...|144324018;4811408...|c64ba547ae3ac0b53...|
|xolw2lti|10.2196/jmir.7.4.e46|Carlos Acosta Riz...|144324018;4811408...|c64ba547ae3ac0b53...|
|7v1ng0ne|10.1146/annurev.m...|Danuta M. Skowron...|6135438;5252414;3...|7b4d551e70ca2ddbb...|
|xjt86q94|10.1146/annurev.m...|Danuta M. Skowron..

In [36]:
val SSmatched = meta2.groupBy("doi").agg(concat_ws(";", collect_list("SSdoi")) as "SSdois",
                                            first("SSauthorNames") as "SSauthorNames",
                                            first("SSauthorIDs") as "SSauthorIDs",
                                            first("SSrefs") as "SSrefs")

SSmatched.describe().filter($"summary" === "count").show

+-------+---+------+
|summary|UID|SSdois|
+-------+---+------+
|  count|315|   315|
+-------+---+------+



SSmatched = [UID: string, SSdois: string ... 3 more fields]


[UID: string, SSdois: string ... 3 more fields]

In [37]:
val SSmatched = meta2.groupBy("cord_uid").agg(concat_ws(";", collect_list("SSdoi")) as "SSdois",
                                            first("SSauthorNames") as "SSauthorNames",
                                            first("SSauthorIDs") as "SSauthorIDs",
                                            first("SSrefs") as "SSrefs")

SSmatched.describe().filter($"summary" === "count").show

+-------+--------+------+
|summary|cord_uid|SSdois|
+-------+--------+------+
|  count|    4989|  4989|
+-------+--------+------+



SSmatched = [cord_uid: string, SSdois: string ... 3 more fields]


[cord_uid: string, SSdois: string ... 3 more fields]

In [39]:
val SSmatched2 = SSmatched.withColumn("SSauthorNameList", array_join($"SSauthorNames", ";"))
                        .withColumn("SSauthorIDList", array_join(flatten($"SSauthorIDs"), ";"))
                        .withColumn("SSrefList", array_join($"SSrefs", ";"))
                        .select("cord_uid","SSdois","SSauthorNameList","SSauthorIDList","SSrefList")
SSmatched2.describe().filter($"summary" === "count").show

+-------+--------+------+----------------+--------------+---------+
|summary|cord_uid|SSdois|SSauthorNameList|SSauthorIDList|SSrefList|
+-------+--------+------+----------------+--------------+---------+
|  count|    4989|  4989|            2727|          2727|     2727|
+-------+--------+------+----------------+--------------+---------+



SSmatched2 = [cord_uid: string, SSdois: string ... 3 more fields]


lastException: Throwable = null


[cord_uid: string, SSdois: string ... 3 more fields]

In [33]:
val meta2 = meta0.join(SSmatched2, Seq("UID"), "left").cache()
meta2.describe().filter($"summary" === "count").show

Name: Unknown Error
Message: lastException: Throwable = null
<console>:46: error: not found: value SSmatched2
       val meta2 = meta0.join(SSmatched2, Seq("UID"), "left").cache()
                              ^

StackTrace: 

In [58]:
meta2.drop("_c0","SSAuthors","SSRefs")
    .coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true")
    .option("quote", "\"")
    .save("wasbs://mag-2020-05-29@iunimag.blob.core.windows.net/2020-06-08/metaAuthors.csv")

In [68]:
val url = "https://api.semanticscholar.org/v1/paper/d1aafb70c066a2068b02786f8929fd9c900897fb"
val result = scala.io.Source.fromURL(url).mkString
println(result)

{"abstract":"ObjectiveThis retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia.MethodsPatients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed.Results40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (35%) with upper respiratory tract infections, and 2 (5%) with bronchiolitis. Cough (82.5%), fever (75%), and malaise (58.8%) w

url = https://api.semanticscholar.org/v1/paper/d1aafb70c066a2068b02786f8929fd9c900897fb
result = 


"{"abstract":"ObjectiveThis retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia.MethodsPatients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed.Results40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred yea...
